# Mixture calculations in a pandapipes network

Since pandapipes release v0.8.0, it is possible to conduct stationary mixture calculations, i.e. different fluids can be fed into a pandapipes grid and the fluid composition can be determined at each junction, respectivaley. With a simple example, we want to visualize the implementation.

The simple example consists of four junctions, one external grid, two sources and three sinks. Both sources feed in hydrogen.

First, we create an empty network and afterwards define the four junctions. Different from the versions before: you don't define one fluid for the entire grid. The nominal pressure is 0.1 bar and the nominal temperature 283.15 K.

In [1]:
import pandapipes as pps
net = pps.create_empty_network()

j0 = pps.create_junction(net, 0.1, 283.15)
j1 = pps.create_junction(net, 0.1, 283.15)
j2 = pps.create_junction(net, 0.1, 283.15)
j3 = pps.create_junction(net, 0.1, 283.15)

hp.pandapower.plotting.plotly.traces - INFO: Failed to import plotly - interactive plotting will not be available


The junctions are connected to each other resembling a t-cross network system. Each pipe has a lenght of 400 m and a diameter of 0.1 m. The other values use the default value.

In [2]:
p1 = pps.create_pipe_from_parameters(net, j0, j1, 0.4, 0.1)
p2 = pps.create_pipe_from_parameters(net, j1, j2, 0.4, 0.1)
p3 = pps.create_pipe_from_parameters(net, j1, j3, 0.4, 0.1)

Next, we define the external grid. Different to the versions before, you need to define a fluid, fed in by the external grid. The external grid is connected to junction 0.

In [5]:
ext = pps.create_ext_grid(net, j0, 'hgas', 0.1, 283.15)

In the following step, we connect the three sinks to junction 1,2, and 3. The mass demand is 0.001, 0.002 and 0.003 kg per s. The sinks are connected to junction 1,2,3.

In [6]:
si0 = pps.create_sink(net, j1, 0.001)
si1 = pps.create_sink(net, j2, 0.002)
si2 = pps.create_sink(net, j3, 0.003)

Conducting a pipeflow leads to following results:

In [7]:
pps.pipeflow(net)

The results look as expected showing no other components besides hgas.

In [8]:
net.res_junction

,p_bar,t_k,rho_kg_per_m3
0,0.100000,283.15,0.751018
1,0.099497,283.15,0.750678
2,0.099410,283.15,0.750619
3,0.099336,283.15,0.750569


In [9]:
net.res_pipe

,v_from_m_per_s,v_to_m_per_s,v_mean_m_per_s,p_from_bar,p_to_bar,t_from_k,t_to_k,mdot_from_kg_per_s,mdot_to_kg_per_s,vdot_norm_m3_per_s,reynolds,lambda,normfactor_from,normfactor_to
0,1.017211,1.017672,1.017441,0.100000,0.099497,283.15,283.15,0.006,-0.006,0.008488,7133.391269,0.032367,0.941184,0.941610
1,0.339224,0.339251,0.339237,0.099497,0.099410,283.15,283.15,0.002,-0.002,0.002829,2377.797090,0.050310,0.941610,0.941684
2,0.508836,0.508910,0.508873,0.099497,0.099336,283.15,283.15,0.003,-0.003,0.004244,3566.695635,0.041339,0.941610,0.941746


You can also define another fluid, which will be listed in **net.fluid**. However, as long it is not definied as fluid for an external grid or a source, it doesn't anything in the results.

In [10]:
pps.create_fluid_from_lib(net, 'lgas')
net.fluid

{'hgas': Fluid hgas (gas) with properties:
    - density (InterExtra)
    - viscosity (InterExtra)
    - heat_capacity (InterExtra)
    - molar_mass (Constant)
    - compressibility (Linear)
    - der_compressibility (Constant)
    - lhv (Constant)
    - hhv (Constant)
    - critical_data (Constant),
 'lgas': Fluid lgas (gas) with properties:
    - density (InterExtra)
    - viscosity (InterExtra)
    - heat_capacity (InterExtra)
    - molar_mass (Constant)
    - compressibility (Linear)
    - der_compressibility (Constant)
    - lhv (Constant)
    - hhv (Constant)
    - critical_data (Constant)}

In [11]:
pps.pipeflow(net)
net.res_junction

,p_bar,t_k,rho_kg_per_m3
0,0.100000,283.15,0.751018
1,0.099497,283.15,0.750678
2,0.099410,283.15,0.750619
3,0.099336,283.15,0.750569


A glimpse at the internal results reveals the number of iterations:

In [12]:
net._internal_results

{'iterations': 3,
 'error_p': 7.404491962981558e-12,
 'error_v': 0.0,
 'error_m': 0.0,
 'error_w': 0,
 'residual_norm': 2.267192045621032e-12}

To see an effect of different fluids being fed into a network, we define two sources. One feeds in 0.001 kg per s, the other one 0.004 kg per s. They are connected to junction 2 and 3. 

In [13]:
so0 = pps.create_source(net, j2, 0.001, fluid='hydrogen')
so1 = pps.create_source(net, j3, 0.004, fluid='hydrogen')

Conducting a pipeflow changes the results as following:

In [14]:
pps.pipeflow(net, iter=100)
net.res_junction

,p_bar,t_k,rho_kg_per_m3,w_hgas,w_hydrogen,rho_kg_per_m3_hgas,rho_kg_per_m3_hydrogen
0,0.100000,283.15,0.747210,1.00,0.00,0.751018,0.090643
1,0.099967,283.15,0.161867,0.50,0.50,0.750995,0.090640
2,0.099828,283.15,0.116279,0.25,0.75,0.750901,0.090629
3,0.100205,283.15,0.090763,0.00,1.00,0.751156,0.090659


At junction 0, we have pure hgas. This makes sense, as the summed up mass of all sources are solely 0.005 kg per s, while sinks sum up to 0.006 kg per s.

At junction 3, we have a composition of 100 % hydrogen, as more hydrogen is fed in compared to the entire load consumed. The difference is 0.001 kg per s, which are fed back to the network system. 

At junction 1, 0.001 kg per s are delivered by the external grid and another 0.001 kg per s come from the feeder leading to junction 3. Therefore, the composition at junction 1 is 50 % hydrogen and 50 % hgas. 

At junction 2, 0.001 kg per s are covered by the source directly connected to junction 2, while the other 0.001 kg per s are supplied through the network system. As junction 1 is directly connected to junction 2, the composition of that 0.001 kg per s are 50 % hydrogen and 50 % hgas. Therefore, in total 0.0015 kg per s hydrogen and 0.005 kg per s hgas are fed into junction 2 and consumed by sink 2, leaving a composition of 75 % hydrogen and 25 % hgas. 

Conclusively, pandapipes is calculating correctly and does this with an additional calcualation overhead of just two more iterations as seen in the internal results.

In [15]:
net._internal_results

{'iterations': 4,
 'error_p': 9.798646416862713e-05,
 'error_v': 5.340494598275943e-16,
 'error_m': 2.168404344971009e-19,
 'error_w': 2.3926610206402113e-16,
 'residual_norm': 1.8208355855424462e-05}

Moreover, in **net.fluid** you can also find hydrogen, as automatically fluids defined in the library are added to the fluid list, if called the first time in an external grid or source.

In [16]:
net.fluid

{'hgas': Fluid hgas (gas) with properties:
    - density (InterExtra)
    - viscosity (InterExtra)
    - heat_capacity (InterExtra)
    - molar_mass (Constant)
    - compressibility (Linear)
    - der_compressibility (Constant)
    - lhv (Constant)
    - hhv (Constant)
    - critical_data (Constant),
 'lgas': Fluid lgas (gas) with properties:
    - density (InterExtra)
    - viscosity (InterExtra)
    - heat_capacity (InterExtra)
    - molar_mass (Constant)
    - compressibility (Linear)
    - der_compressibility (Constant)
    - lhv (Constant)
    - hhv (Constant)
    - critical_data (Constant),
 'hydrogen': Fluid hydrogen (gas) with properties:
    - density (InterExtra)
    - viscosity (InterExtra)
    - heat_capacity (InterExtra)
    - molar_mass (Constant)
    - compressibility (Linear)
    - der_compressibility (Constant)
    - lhv (Constant)
    - hhv (Constant)
    - critical_data (Constant)}

So, that's it for now. If you have any recommendations or find bugs, please get in touch with us through our issue board on https://github.com/e2nIEE/pandapipes/issues